In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### pandas와 모델 코드
### 변수 엔지니어링

In [2]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


- 데이터 프레임을 NumPy 배열로 변환하려면 .values 속성 사용

In [3]:
data.columns, data.values

(Index(['x0', 'x1', 'y'], dtype='object'),
 array([[ 1.  ,  0.01, -1.5 ],
        [ 2.  , -0.01,  0.  ],
        [ 3.  ,  0.25,  3.6 ],
        [ 4.  , -4.1 ,  1.3 ],
        [ 5.  ,  0.  , -2.  ]]))

- 다시 데이터 프레임으로 되돌림
- **.values 속성은 데이터가 한 가지 타입으로 이루어져 있다는 가정하에 사용**
- 만약 데이터 속성이 한 가지가 아니라면 파이썬 객체의 ndarray가 반환

In [4]:
df2 = pd.DataFrame(data.values, columns = ['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


#### 전체 열 중 일부만 사용 시

In [5]:
model_cols = ['x0', 'x1']

In [6]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [7]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                  categories = ['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


#### 'category'열을 더미변수화

In [9]:
dummies = pd.get_dummies(data.category, prefix = 'category')
data_with_dummies = data.drop('category', axis = 'columns').join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


### 
### Patsy
- Patsy(팻시)는 통계 모형을 위한 파이썬 라이브러리
- R 등의 언어에서 사용하는 수식 문법과 비슷한 형식의 '수식 문법'제공

In [10]:
import patsy

In [11]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


### ```patsy.dmatrices('수식', data)``` : 선형 모형 설계 배열

In [12]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

In [14]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [18]:
np.asarray(y), np.asarray(X)

(array([[-1.5],
        [ 0. ],
        [ 3.6],
        [ 1.3],
        [-2. ]]),
 array([[ 1.  ,  1.  ,  0.01],
        [ 1.  ,  2.  , -0.01],
        [ 1.  ,  3.  ,  0.25],
        [ 1.  ,  4.  , -4.1 ],
        [ 1.  ,  5.  ,  0.  ]]))

- Intercept는 최소제곱법과 같은 선형 모형을 위한 표현
- 모델에 0을 더해서 intercept(절편)를 제거 가능

In [19]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

- **Patsy객체는 최소죄곱회귀분석을 위해 ```numpy.linalg.lstsq``` 같은 알고리즘에 바로 넘길 수 있다.**

In [20]:
coef, resid, _, _ = np.linalg.lstsq(X, y)
coef

C:\Users\이찬솔\AppData\Local\Temp\ipykernel_10128\1708406644.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, resid, _, _ = np.linalg.lstsq(X, y)


array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

#### 회귀 계수

In [21]:
coef = pd.Series(coef.squeeze(), index = X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### 
### Patsy 용법으로 데이터 변환

In [23]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

- **표준화(평균 0, 분산 1), 센터링 (평균값을 차감)**

In [24]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

### 
### **홀드 아웃** 혹은 신규 데이터
- 센터링이나 표준화 같은 변환을 적용하는 경우, 새롱 넫이터에 기반하여 예측하기 위한 용도로 모델을 사용한다면 주의 필요

### ```patsy.build_design_matrices()``` : 입력으로 사용되는 원본 데이터셋에서 저장한 정보를 사용해서 출력 데이터를 만들어내는 변환에 적용

In [25]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [3.1, -0.5, 0, 2.3],
    'y': [1, 2, 3, 4]})

- **X에 적용한 표준화, 센터링을, 새로운 데이터에 적용**

In [26]:
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [27]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### 
### 범주형 데이터와 Patsy

In [28]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v1': [1, 2, 3, 4, 5, 6, 7, 8],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

In [29]:
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

- **모형에서 intercept를 생략하면 각 범주값의 열은 모형 설계 배열에 포함**

In [30]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

- **산술 열은 C 함수를 이용해서 범주형으로 해석**

In [31]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

### 
#### 모형에서 여러 범주형 항을 사용한다면, ANOVA 모델에서 처럼 key1 : key2 같은 용법 사용

In [32]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [33]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [34]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

### 
## statsmodels
- 선형 모형, 일반 선형 모형, 로버스트 선형 모형
- 선형 복합효과 (Linear Mixed Effects, LME)
- ANOVA
- 시계열 처리 및 상태 공간 모델
- 일반적률추정법 (GMM)

### 선형 모형 예측

In [35]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

- **랜덤 데이터에서 선형 모형 생성**

In [37]:
def dnorm(mean, variance, size = 1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [38]:
np.random.seed(12345)

N = 100
X = np.c_[dnorm(0, 0.4, size = N),
          dnorm(0, 0.6, size = N),
          dnorm(0, 0.2, size = N)]
eps = dnorm(0, 0.1, size = N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [39]:
X[:5], y[:5]

(array([[-0.12946849, -1.21275292,  0.50422488],
        [ 0.30291036, -0.43574176, -0.25417986],
        [-0.32852189, -0.02530153,  0.13835097],
        [-0.35147471, -0.71960511, -0.25821463],
        [ 1.2432688 , -0.37379916, -0.52262905]]),
 array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109]))

### ```sm.add_constant()``` : intercept 열을 기존 행렬에 추가

In [40]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

### ```sm.OLS(y, X)``` : 최소제곱 선형회귀에 적합

In [41]:
model = sm.OLS(y, X)

### ```model.fit()``` : 예측 모델 인자와, 다른 분석 정보를 포함하는 회귀 결과 객체를 반환

In [44]:
results = model.fit()
results.params

array([0.17826108, 0.22303962, 0.50095093])

In [47]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Tue, 09 Aug 2022   Prob (F-statistic):                    7.44e-12
Time:                        05:25:27   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## 

- statsmodels의 API와 Patsy의 문자열 용법 사용

In [53]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data.head()

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


In [49]:
results = smf.ols('y ~ col0 + col1 + col2', data = data).fit()

In [55]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.435
Model:                            OLS   Adj. R-squared:                  0.418
Method:                 Least Squares   F-statistic:                     24.68
Date:                Tue, 09 Aug 2022   Prob (F-statistic):           6.37e-12
Time:                        05:28:02   Log-Likelihood:                -33.835
No. Observations:                 100   AIC:                             75.67
Df Residuals:                      96   BIC:                             86.09
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0336      0.035      0.952      0.3

### ```results.predict(data)``` : 예측

In [56]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

### 
### 시계열 처리 예측

- 자동회귀 모형

### ```sm.tsa.AR()```

In [63]:
import random

In [64]:
init_x = 4

In [65]:
values = [init_x, init_x]
N = 1000

In [66]:
b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)

In [67]:
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [71]:
MAXLAGS = 5

```Python
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)
results.params
```

## 
### scikit-learn

In [73]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

- 결측치 확인

In [76]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [77]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

- 결측치를 중간값으로 대체

In [78]:
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

- 'IsFemale'열을 추가

In [79]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [82]:
train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsFemale
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1


- 몇 가지 모형 변수를 선언, NumPy 배열 생성

In [88]:
predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values

In [89]:
y_train = train['Survived'].values

In [95]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [97]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

### 
#### scikit-learn의 LogisticRegression : 로지스틱 회귀
### ```LogisticRegression()``` : 로지스틱 회귀 객체 생성

In [98]:
from sklearn.linear_model import LogisticRegression

In [102]:
model = LogisticRegression()

In [103]:
model.fit(X_train, y_train)

LogisticRegression()

In [105]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

### 교차검증

In [106]:
from sklearn.linear_model import LogisticRegressionCV

```Python
model_cv = LogisticRegressionCV(10)
model_cv.fit(X_train, y_train)
```

### 직접 교차검증

### ```cross_val_score(모형, X_train, y_train, cv)```

In [110]:
from sklearn.model_selection import cross_val_score

In [111]:
model = LogisticRegression(C = 10)

- 학습 데이터를 겹치지 않는 10개의 그룹으로 분할

In [113]:
scores = cross_val_score(model, X_train, y_train, cv = 10)
scores

array([0.78888889, 0.79775281, 0.74157303, 0.83146067, 0.80898876,
       0.78651685, 0.78651685, 0.76404494, 0.82022472, 0.79775281])